In [ ]:
# TODO: Make basebrowser class

# Reading Chrome Bookmarks from all profiles
> This module reads the Chrome Bookmarks from all profiles and returns a list of bookmarks.


In [ ]:
#|hide
#|default_exp bookmarks.browser.chrome_bookmarks

In [ ]:
#|hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#|export
import json
import os
import sys
from datetime import date, datetime, timedelta
from pathlib import Path

from fastcore.all import patch
from pydantic import BaseModel, Field


In [ ]:
#|export
class ChromeBookmarks:
    """
    A class to interact with Chrome Bookmarks.
    """

    user_data_dir_paths = {
        "Windows": r"~\AppData\Local\Google\Chrome\User Data",
        "Mac": r"~\Library\Application Support\Google\Chrome",
        "Linux": r"~\.config\google-chrome",
    }

    def __init__(self):
        self.operating_system = None
        if "win" in sys.platform.lower():
            self.operating_system = "Windows"
        elif "mac" in sys.platform.lower():
            self.operating_system = "Mac"
        elif "linux" in sys.platform.lower():
            self.operating_system = "Linux"
        else:
            self.operating_system = "Unknown"
        self.user_data_dir_path = os.path.expanduser(
            self.user_data_dir_paths[self.operating_system]
        )


In [ ]:
#|export
@patch
def get_chrome_bookmarks_files(self: ChromeBookmarks):
    chrome_path = Path(self.user_data_dir_path)
    # find all bookmarks files in chrome_path
    bookmarks_files = list(chrome_path.glob("**/*Bookmarks"))
    # Ignore the snapshot files
    bookmarks_files = [
        file.as_posix() for file in bookmarks_files if "Snapshot" not in file.as_posix()
    ]
    return bookmarks_files

In [ ]:
#|export
class BookmarkModel(BaseModel):
    """
    A model representing a bookmark.
    """

    name: str = Field(..., description="The name of the bookmark")
    url: str = Field(..., description="The URL of the bookmark")
    folder: str = Field(..., description="The folder of the bookmark")
    date_added: date = Field(..., description="The date the bookmark was added")

    def __str__(self):
        return f"{self.name} ({self.url}) added on {self.date_added}"

    __repr__ = __str__

In [ ]:
#|export
def windows_epoch_to_datetime(windows_timestamp):
    """Convert Windows epoch timestamp to datetime object."""
    windows_epoch = datetime(1601, 1, 1)
    delta = timedelta(microseconds=int(windows_timestamp))
    return (windows_epoch + delta).date()

In [ ]:
#|export
@patch
def extract_bookmark_info(
    self: ChromeBookmarks,
    bookmarks_file_path: str,  # path to the bookmarks file
) -> list[BookmarkModel]:
    """Extracts bookmark information from a Chrome Bookmarks file."""
    with open(bookmarks_file_path, "r", encoding="utf-8") as f:
        bookmarks_data = json.load(f)

    bookmarks = []

    def traverse_bookmarks(node, folder=None):
        """Recursively traverses the bookmark tree."""
        if node["type"] == "url":
            date_added = windows_epoch_to_datetime(node["date_added"])
            bookmarks.append(
                BookmarkModel(
                    name=node["name"],
                    url=node["url"],
                    folder=folder,
                    date_added=date_added,
                )
            )
        elif node["type"] == "folder":
            for child in node["children"]:
                traverse_bookmarks(child, folder=node["name"])

    for root_name, root_node in bookmarks_data["roots"].items():
        traverse_bookmarks(root_node, folder=root_name)

    return bookmarks

In [ ]:
#|export
@patch
def extract_all_chrome_bookmarks(self: ChromeBookmarks) -> list[BookmarkModel]:
    """Extracts all Chrome Bookmarks from all profiles."""
    bookmarks_files = self.get_chrome_bookmarks_files()
    bookmarks = []
    for bookmarks_file in bookmarks_files:
        bookmarks.extend(self.extract_bookmark_info(bookmarks_file))
    return bookmarks

In [ ]:
#|export
if __name__ == "__main__":
    chrome_bookmarks = ChromeBookmarks()
    bookmarks = chrome_bookmarks.extract_all_chrome_bookmarks()
    print(bookmarks)

'C:\\Users\\vedmani.vaidya\\AppData\\Local\\Google\\Chrome\\User Data'
[Introducing the Hugging Face LLM Inference Container for Amazon SageMaker (https://huggingface.co/blog/sagemaker-huggingface-llm) added on 2024-05-17, Serving LLMs using HuggingFace and Kubernetes on OCI (https://blogs.oracle.com/ai-and-datascience/post/serving-llm-using-huggingface-and-kubernetes-oci) added on 2024-05-17, Deployment | 🦜️🔗 LangChain (https://python.langchain.com/v0.1/docs/guides/productionization/deployments/) added on 2024-05-17, Hosting any LLaMA 3 model with Text Generation Inference (TGI) | Modal Docs (https://modal.com/docs/examples/text_generation_inference) added on 2024-05-17, Deploying LLMs Into Production Using TensorRT LLM | by Het Trivedi | Towards Data Science (https://towardsdatascience.com/deploying-llms-into-production-using-tensorrt-llm-ed36e620dac4) added on 2024-05-17, Deploying LLM Models: A Comprehensive Guide to Development and Deployment of Large Language Model Applications i